In [1]:
from tqdm import tqdm
import time
import os
import pandas as pd
import warnings 
warnings.filterwarnings("ignore")

In [6]:
data=pd.read_csv('/Volumes/T7/mt-hemingway/M2M100_de_en_ko.csv', encoding='utf-8-sig')

In [7]:
data.columns

Index(['Unnamed: 0', 'small', 'big', 'OPUS_to_DE_small', 'OPUS_to_DE_big',
       'M2M100_to_EN_small', 'M2M100_to_EN_big'],
      dtype='object')

In [8]:
data.head(2)

,Unnamed: 0,small,big,OPUS_to_DE_small,OPUS_to_DE_big,M2M100_to_EN_small,M2M100_to_EN_big
0,0,산티아고는 멕시코 만류에서 조각배를 타고 혼자 고기잡이를 하는 늙은 어부였다. 오늘...,그는 멕시코 해류에서 조각배를 타고 홀로 고기잡이하는 노인이었다. 여든 날하고도 나...,"Santiago war ein alter Fischer, der allein in ...","Er war ein alter Mann, der allein mit einem Bo...",Santiago was an old fisherman who went fishing...,He was an old man who was fishing alone with a...
1,1,처음 40일 동안은 한 소년과 함께 있었다. 그러나 40일이 지나도록 물고기를 잡지...,처음 사십 일 동안은 소년이 함께 있었다. 그러나 사십 일이 지나도록 고기 한 마리...,In den ersten 40 Tagen war er mit einem Jungen...,Als aber 40 Tage später kein Fleisch mehr da w...,"In the first 40 days he was with a boy, but af...","But 40 days later, when no meat was there, his..."


In [4]:
small_big=data[['small', 'big']]

OPUS

In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [6]:
it_en_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ko-de")
it_en_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ko-de")

In [7]:
def OPUS_to_DE(data: pd.DataFrame, column: str): 
    translated_small=[]
    for idx, val in tqdm(data.iterrows()):
        for_translate=val[column]
        try:
            encoded_input=it_en_tokenizer(for_translate, return_tensors='pt')
            output=it_en_model.generate(**encoded_input)
            out_text=it_en_tokenizer.batch_decode(output, skip_special_tokens=True)
            out_text=out_text[0]
            out_text=out_text.replace('<pad>', '')
            # time.sleep(0.5)
            translated_small.append(out_text)
        except TypeError:
            translated_small.append(val)
            # time.sleep(0.5)
            continue
    data['OPUS_to_DE'+'_'+column]=translated_small
    return data

In [8]:
small_de=OPUS_to_DE(small_big, 'small')

768it [13:48,  1.08s/it]


In [9]:
big_de=OPUS_to_DE(small_de, 'big')

768it [14:11,  1.11s/it]


In [10]:
de_en_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-de-en")#, model_max_length=512)
de_en_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-de-en")

In [11]:
def OPUS_to_EN(data: pd.DataFrame, column: str): 
    translated_small=[]
    for idx, val in tqdm(data.iterrows()):
        for_translate=val[column]
        try:
            encoded_input=de_en_tokenizer(for_translate, return_tensors='pt')
            output=de_en_model.generate(**encoded_input)
            out_text=it_en_tokenizer.batch_decode(output, skip_special_tokens=True)
            out_text=out_text[0]
            out_text=out_text.replace('<pad>', '')
            # time.sleep(0.5)
            translated_small.append(out_text)
        except TypeError:
            translated_small.append(val)
            # time.sleep(0.5)
            continue
    data['OPUS_to_EN'+'_'+column]=translated_small
    return data

In [22]:
small_EN2=OPUS_to_EN(big_de.head(2), 'OPUS_to_DE_small')

2it [00:02,  1.05s/it]


In [16]:
big_EN=OPUS_to_EN(small_EN, 'OPUS_to_DE_big')

768it [09:03,  1.41it/s]


In [ ]:
en_ko_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-ko")
en_ko_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-ko")

In [ ]:
def OPUS_to_KO(data: pd.DataFrame, column: str): 
    translated_small=[]
    for idx, val in tqdm(data.iterrows()):
        for_translate=val[column]
        try:
            encoded_input=en_ko_tokenizer(for_translate, return_tensors='pt')
            output=it_en_model.generate(**encoded_input)
            out_text=en_ko_tokenizer.batch_decode(output, skip_special_tokens=True)
            out_text=out_text[0]
            out_text=out_text.replace('<pad>', '')
            # time.sleep(0.5)
            translated_small.append(out_text)
        except TypeError:
            translated_small.append(val)
            # time.sleep(0.5)
            continue
    data['OPUS_to_KO'+'_'+column]=translated_small
    return data

In [ ]:
small_KO=OPUS_to_KO(big_EN, 'OPUS_to_EN_small')

In [ ]:
big_KO=OPUS_to_KO(small_KO, 'OPUS_to_EN_big')

In [21]:
small_EN.to_csv('OPUS_de_en_ko.csv', encoding='utf-8-sig')

In [31]:
small_EN.drop(['OPUS_to_EN_OPUS_to_DE_small', 'OPUS_to_EN_OPUS_to_DE_big'], axis=1, inplace=True)

M2M100

In [12]:
import numpy as np
from transformers import M2M100Config, M2M100ForConditionalGeneration, M2M100Tokenizer

model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M", src_lang="en", tgt_lang="ko")

In [32]:
small_EN

,small,big,OPUS_to_DE_small,OPUS_to_DE_big
0,산티아고는 멕시코 만류에서 조각배를 타고 혼자 고기잡이를 하는 늙은 어부였다. 오늘...,그는 멕시코 해류에서 조각배를 타고 홀로 고기잡이하는 노인이었다. 여든 날하고도 나...,"Santiago war ein alter Fischer, der allein in ...","Er war ein alter Mann, der allein mit einem Bo..."
1,처음 40일 동안은 한 소년과 함께 있었다. 그러나 40일이 지나도록 물고기를 잡지...,처음 사십 일 동안은 소년이 함께 있었다. 그러나 사십 일이 지나도록 고기 한 마리...,In den ersten 40 Tagen war er mit einem Jungen...,Als aber 40 Tage später kein Fleisch mehr da w...
2,소년은 허구한 날 빈 배로 돌아오는 노인의 모습에 가슴이 아팠다. 그래서 소년은 늘...,소년은 날마다 노인이 빈 배로 돌아오는 것을 보고 가슴이 아팠다. 그래서 늘 노인을...,Der Junge war von dem Anblick des alten Mannes...,"Der Junge war traurig, als er jeden Tag einen ..."
3,밀가루 부대 조각 따위로 여기저기 더덕더덕 기워진 돛은 둘둘 말면 마치 영원한 패배...,"돛은 여기저기 밀가루 부대로 기워져 있었고, 접어 놓으면 마치 영원한 패배를 상징하...","Es sah aus wie eine Flagge, die eine ewige Nie...",Die Segel waren von Mehl überwuchert und sahen...
4,산티아고는 깡마른 몸에 목덜미에는 깊은 주름살이 패어 있었다.,노인은 깡마르고 여윈 데다 목덜미에는 주름이 깊게 잡혀 있었다.,Santiago hatte tiefe Falten in seinem trockene...,"Der alte Mann war trocken, und er hatte eine t..."
...,...,...,...,...
763,“저게 뭐예요?” 부인이 안내원에게 물었다. 그때 마침 고기의 뼈는 조류를 타고 바...,저게 뭐죠? 여자가 웨이터에게 물으면서 이제 해류를 타고 바다로 밀려 나가기를 기다...,"Mrs. Wong fragte den Lotsen, als die Knochen d...","Eine Frau fragte einen Kellner, und sie zeigte..."
764,티뷰론입니다. 안내원이 대답했다. “상어의 일종이죠.” 안내원은 사투리가 섞인 영어...,티부론 '이죠. 상어랍니다. 웨이터가 대답했다. 그러면서 그는 사건의 경위를 설명하...,"Der Lotse antwortete, es war eine Art Hai, der...","Tiburon, ein Hai, antwortete der Kellner, und ..."
765,"“세상에, 상어가 저렇게 멋있고 아름다운 꼬리를 가지고 있을 줄은 몰랐어요.”",상어가 저토록 잘생기고 멋진 꼬리를 달고 있는 줄은 미처 몰랐어요.,"Ich hätte nie gedacht, dass ein Hai einen so s...","Ich wusste nicht, dass der Hai einen schönen, ..."
766,"“정말이야, 나도 처음 본걸.” 곁에 있던 부인의 동행인 남자가 말했다.",나도 몰랐는걸. 여자와 동행인 남자가 말했다.,"Der Mann, der die Frau begleitete, sagte:","Der Mann, der mit der Frau unterwegs war, sagte:"


In [46]:
def M2M100_to_EN(df:pd.DataFrame, column:str):
    translated=[]
    tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M", src_lang="de", tgt_lang="en")
    for i in tqdm(df[column]):
        try:
            encoded_hi=tokenizer(i, return_tensors='pt')
            generated_tokens = model.generate(**encoded_hi, forced_bos_token_id=tokenizer.get_lang_id("en"))
            translated.append(tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0])
        except ValueError:
            translated.append(np.nan)
            # time.sleep(0.5)
            continue
    df['M2M100_to_EN'+'_'+column.replace('OPUS_to_DE_', '')]=translated
    return df

In [47]:
small_m2m100_EN=M2M100_to_EN(small_EN, 'OPUS_to_DE_small')

100%|██████████| 768/768 [41:23<00:00,  3.23s/it]  


In [58]:
big_m2m100_EN=M2M100_to_EN(small_m2m100_EN, 'OPUS_to_DE_big')

100%|██████████| 768/768 [41:09<00:00,  3.21s/it]


In [13]:
def M2M100_to_KO(df:pd.DataFrame, column:str):
    translated=[]
    tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M", src_lang="en", tgt_lang="ko")
    for i in tqdm(df[column]):
        try:
            encoded_hi=tokenizer(i, return_tensors='pt')
            generated_tokens = model.generate(**encoded_hi, forced_bos_token_id=tokenizer.get_lang_id("ko"))
            translated.append(tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0])
        except ValueError:
            translated.append(np.nan)
            # time.sleep(0.5)
            continue
    df['M2M100_to_KO'+'_'+column.replace('M2M100_to_EN_', '')]=translated
    return df

In [15]:
small_m2m100_KO=M2M100_to_KO(data, 'M2M100_to_EN_small')

100%|██████████| 768/768 [45:48<00:00,  3.58s/it] 


In [16]:
big_m2m100_KO=M2M100_to_KO(small_m2m100_KO, 'M2M100_to_EN_big')

100%|██████████| 768/768 [45:58<00:00,  3.59s/it] 


In [18]:
big_m2m100_KO.to_csv('OPUS_M2M100.csv', index=False, encoding='utf-8-sig')